In [1]:
import pandas as pd
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt

In [2]:
data = pd.read_csv("heart_cleveland_upload.csv")

In [3]:
class Dataset(torch.utils.data.Dataset):
    def __init__(self, csv_file):
        self.data = pd.read_csv(csv_file)
        self.x = self.data.iloc[:, :-1].values.astype('float32')
        self.y = self.data.iloc[:, -1].values.astype('float32')

        self.scaler = StandardScaler()
        self.x = self.scaler.fit_transform(self.x).astype('float32')

    def __len__(self):
        return len(self.x)
    def __getitem__(self, index):
        return torch.tensor(self.x[index]),  torch.tensor(self.y[index])

In [4]:
dataset = Dataset("heart_cleveland_upload.csv")

In [5]:
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size

train_data, test_data = torch.utils.data.random_split(dataset = dataset,
                            lengths = [train_size, test_size],
                            generator = torch.Generator().manual_seed(17))

In [6]:
train_loader = torch.utils.data.DataLoader(dataset = train_data,
                            batch_size = 25,
                            shuffle = True)

test_loader = torch.utils.data.DataLoader(dataset = test_data,
                            batch_size = 25,
                            shuffle = False)

In [15]:
class Model(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(13, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 16)
        self.fc4 = nn.Linear(16, 1)
        self.tanh = nn.Tanh()
        self.sigmoid = nn.Sigmoid()
        self.dropout = nn.Dropout(0.5)
    def forward(self, x):
        out = self.fc1(x)
        out = self.tanh(out)
        out = self.dropout(out)

        out = self.fc2(out)
        out = self.tanh(out)
        out = self.dropout(out)

        out = self.fc3(out)
        out = self.tanh(out)
        out = self.dropout(out)

        out = self.fc4(out)
        out = self.sigmoid(out)
        return out 

In [17]:
network = Model()
criterion = torch.nn.BCELoss(reduction = 'mean')
optimalizer = torch.optim.Adam(network.parameters(), lr = 0.001)

num_epoch = 500

for epoch in range(num_epoch):
    network.train()
    
    running_loss = 0.0
    correct_train = 0
    
    for (x, y) in train_loader:
        y = y.unsqueeze(1)

        output = network(x)
        loss = criterion(output, y)

        correct_train += ((output >= 0.5).int() == y).sum().item()
        running_loss += loss.item() * y.size(0)

        optimalizer.zero_grad()
        loss.backward()
        optimalizer.step()
    avg_loss = running_loss / len(train_data)
    train_acc = correct_train / len(train_data)

    if epoch % 50 == 0:
        network.eval()

        correct_test = 0

        with torch.no_grad():
            for i, (x, y) in enumerate(test_loader):
                y = y.unsqueeze(1)
                output = network(x)
                correct_test +=  ((output >= 0.5).int() == y).sum().item()
        test_acc = correct_test / len(test_data)

        print("Epoch: [{}/{}], Loss: {:.4f}, Training Accuracy: {:.4f}, Testing Accuracy: {:.4f}".format(epoch+50,
                                                                                        num_epoch,
                                                                                        avg_loss,
                                                                                        train_acc,
                                                                                        test_acc))

Epoch: [50/500], Loss: 0.6662, Training Accuracy: 0.5907, Testing Accuracy: 0.8000
Epoch: [100/500], Loss: 0.3904, Training Accuracy: 0.8439, Testing Accuracy: 0.8667
Epoch: [150/500], Loss: 0.4116, Training Accuracy: 0.8312, Testing Accuracy: 0.8500
Epoch: [200/500], Loss: 0.3333, Training Accuracy: 0.8608, Testing Accuracy: 0.8500
Epoch: [250/500], Loss: 0.3479, Training Accuracy: 0.8650, Testing Accuracy: 0.8833
Epoch: [300/500], Loss: 0.3509, Training Accuracy: 0.8692, Testing Accuracy: 0.8833
Epoch: [350/500], Loss: 0.3399, Training Accuracy: 0.8565, Testing Accuracy: 0.8833
Epoch: [400/500], Loss: 0.2787, Training Accuracy: 0.8903, Testing Accuracy: 0.8833
Epoch: [450/500], Loss: 0.2685, Training Accuracy: 0.9030, Testing Accuracy: 0.8667
Epoch: [500/500], Loss: 0.2722, Training Accuracy: 0.9030, Testing Accuracy: 0.8667
